## Installation

In [1]:
pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 6.8 MB/s eta 0:00:00


## API Key

In [2]:
from google.colab import userdata
CLAUDE_API_KEY = userdata.get('CLAUDE_API_KEY')

## Define Main Chat Manager

In [3]:
#!/usr/bin/env python3
"""
Fixed version of the Multi-Tool Chat Manager with Pydantic V2 compatibility.
"""

from typing import List, Dict, Any, Optional, Callable, Union, Literal
from abc import ABC, abstractmethod
from datetime import datetime
import time
from pydantic import BaseModel, Field, field_validator, ValidationError


# Pydantic Models for Input Validation
class ArticleSearchInput(BaseModel):
    """Input model for article search tool with validation."""

    search_term: str = Field(
        ...,
        description="The term to search for in Wikipedia",
        min_length=1,
        max_length=100
    )


class WebSearchInput(BaseModel):
    """Input model for web search tool with validation."""

    topic: str = Field(
        ...,
        description="The topic to search the web for",
        min_length=1,
        max_length=100
    )


# Pydantic Models for Output Structure
class ToolResult(BaseModel):
    """Base model for tool execution results."""

    success: bool = Field(..., description="Whether the tool execution was successful")
    content: str = Field(..., description="The main content or result from the tool")
    error_message: Optional[str] = Field(None, description="Error message if execution failed")
    timestamp: datetime = Field(default_factory=datetime.now, description="When the tool was executed")


class ArticleResult(ToolResult):
    """Structured result for article search with additional metadata."""

    article_title: Optional[str] = Field(None, description="The title of the retrieved article")
    word_count: Optional[int] = Field(None, ge=0, description="Number of words in the article")


class WebSearchResult(ToolResult):
    """Structured result for web search with additional metadata."""

    sources_count: Optional[int] = Field(None, ge=0, description="Number of sources found")
    search_query: str = Field(..., description="The actual query used for searching")


# Pydantic Models for Conversation Management
class MessageContent(BaseModel):
    """Base class for message content with type discrimination."""

    type: str = Field(..., description="The type of content")


class TextContent(MessageContent):
    """Text content in a conversation message."""

    type: Literal["text"] = "text"
    text: str = Field(..., description="The text content")


class ToolUseContent(MessageContent):
    """Tool use content in a conversation message."""

    type: Literal["tool_use"] = "tool_use"
    id: str = Field(..., description="ID of the tool use")
    name: str = Field(..., description="Name of the tool being used")
    input: Dict[str, Any] = Field(..., description="Input parameters for the tool")


class ToolResultContent(MessageContent):
    """Tool result content in a conversation message."""

    type: Literal["tool_result"] = "tool_result"
    tool_use_id: str = Field(..., description="ID of the tool use this result corresponds to")
    content: str = Field(..., description="The result content from the tool")


class ConversationMessage(BaseModel):
    """Structured conversation message with validation."""

    role: Literal["user", "assistant", "system"] = Field(..., description="The role of the message sender")
    content: Union[str, List[Union[TextContent, ToolUseContent, ToolResultContent, Dict[str, Any]]]] = Field(
        ..., description="The message content, either string or structured list"
    )
    timestamp: datetime = Field(default_factory=datetime.now, description="When the message was created")

    @field_validator('content')
    @classmethod
    def validate_content(cls, content_value: Any, info) -> Any:
        """
        Validate that system messages have string content.

        Args:
            content_value: The content to validate
            info: Validation info including other field values

        Returns:
            The validated content

        Raises:
            ValueError: If system message doesn't have string content
        """
        # Get the role from the data being validated
        data = info.data if hasattr(info, 'data') else {}
        role = data.get('role')

        if role == 'system' and not isinstance(content_value, str):
            raise ValueError("System messages must have string content")
        return content_value


class ConversationHistory(BaseModel):
    """Complete conversation history with metadata and methods."""

    messages: List[ConversationMessage] = Field(default_factory=list, description="List of conversation messages")
    tool_call_count: int = Field(default=0, ge=0, description="Total number of tool calls made")
    created_at: datetime = Field(default_factory=datetime.now, description="When the conversation was created")

    def add_message(self, message: ConversationMessage) -> None:
        """
        Add a validated message to the conversation history.

        Args:
            message: The validated conversation message to add
        """
        self.messages.append(message)

    def clear(self) -> None:
        """Clear the conversation history and reset all counters."""
        self.messages.clear()
        self.tool_call_count = 0


# Pydantic Models for Configuration
class ToolDefinition(BaseModel):
    """Pydantic model for tool definitions with validation."""

    name: str = Field(..., description="Unique name for the tool")
    description: str = Field(..., description="Human-readable description of the tool")
    input_schema: Dict[str, Any] = Field(..., description="JSON schema defining the tool's input format")

    @field_validator('name')
    @classmethod
    def validate_name(cls, name_value: str) -> str:
        """
        Validate that tool name is alphanumeric with underscores only.

        Args:
            name_value: The tool name to validate

        Returns:
            The validated tool name

        Raises:
            ValueError: If name contains invalid characters
        """
        if not name_value.replace('_', '').isalnum():
            raise ValueError("Tool name must be alphanumeric with underscores only")
        return name_value


class ChatManagerConfig(BaseModel):
    """Configuration model for MultiToolChatManager with validation."""

    model: str = Field(
        default="claude-sonnet-4-20250514",
        description="Claude model identifier to use for conversations"
    )
    max_tokens: int = Field(
        default=1000,
        ge=1,
        le=4000,
        description="Maximum tokens per API response"
    )
    max_tool_iterations: int = Field(
        default=5,
        ge=1,
        le=10,
        description="Maximum number of tool call iterations per conversation turn"
    )
    tools: List[ToolDefinition] = Field(
        default_factory=list,
        description="List of available tool definitions"
    )

    @field_validator('model')
    @classmethod
    def validate_model(cls, model_value: str) -> str:
        """
        Validate that the model is one of the supported Claude models.

        Args:
            model_value: The model identifier to validate

        Returns:
            The validated model identifier

        Raises:
            ValueError: If model is not in the list of valid models
        """
        valid_models = [
            "claude-sonnet-4-20250514",
            "claude-opus-4",
            "claude-haiku-4"
        ]
        if model_value not in valid_models:
            raise ValueError(f"Model must be one of: {valid_models}")
        return model_value


class ChatResponse(BaseModel):
    """Structured response from chat invocation with metrics."""

    content: str = Field(..., description="The main response content from Claude")
    tool_calls_made: int = Field(..., ge=0, description="Number of tool calls made during this interaction")
    success: bool = Field(..., description="Whether the chat interaction was successful")
    error_message: Optional[str] = Field(None, description="Error message if the interaction failed")
    response_time: Optional[float] = Field(None, ge=0, description="Time taken for the response in seconds")
    tokens_used: Optional[int] = Field(None, ge=0, description="Number of tokens used in the API call")


# Abstract Base Classes
class ToolHandler(ABC):
    """Abstract base class for tool handlers with type safety."""

    @abstractmethod
    def handle(self, tool_input: Dict[str, Any]) -> str:
        """
        Handle the execution of a specific tool.

        Args:
            tool_input: Dictionary containing the input parameters for the tool

        Returns:
            str: The result of the tool execution

        Raises:
            ValidationError: If the input doesn't match the expected schema
            Exception: If tool execution fails for any other reason
        """


# Concrete Tool Handler Implementations
class ArticleSearchHandler(ToolHandler):
    """Handler for article search tool with Pydantic validation."""

    def __init__(self, get_article_func: Callable[[str], str]) -> None:
        """
        Initialize the article search handler with a retrieval function.

        Args:
            get_article_func: Function that takes a search term and returns article content
        """
        self.get_article_func = get_article_func

    def handle(self, tool_input: Dict[str, Any]) -> str:
        """
        Handle article search requests with input validation.

        Args:
            tool_input: Dictionary containing 'search_term' key

        Returns:
            str: The retrieved article content

        Raises:
            ValidationError: If input validation fails
            Exception: If article retrieval fails
        """
        # Validate input using Pydantic model
        validated_input = ArticleSearchInput(**tool_input)
        print(f"Invoking tool: ... wait ... Searching for article: {validated_input.search_term}")

        try:
            result = self.get_article_func(validated_input.search_term)
            return result
        except Exception as exc:
            error_msg = f"Failed to retrieve article for '{validated_input.search_term}': {str(exc)}"
            print(error_msg)
            raise Exception(error_msg) from exc


class WebSearchHandler(ToolHandler):
    """Handler for web search tool with Pydantic validation."""

    def __init__(self, web_search_func: Callable[[str], str]) -> None:
        """
        Initialize the web search handler with a search function.

        Args:
            web_search_func: Function that takes a topic and returns search results
        """
        self.web_search_func = web_search_func

    def handle(self, tool_input: Dict[str, Any]) -> str:
        """
        Handle web search requests with input validation.

        Args:
            tool_input: Dictionary containing 'topic' key

        Returns:
            str: The web search results

        Raises:
            ValidationError: If input validation fails
            Exception: If web search fails
        """
        # Validate input using Pydantic model
        validated_input = WebSearchInput(**tool_input)
        print(f"Invoking tool: ... wait ... Searching the web for: {validated_input.topic}")

        try:
            result = self.web_search_func(validated_input.topic)
            return result
        except Exception as exc:
            error_msg = f"Failed to search web for '{validated_input.topic}': {str(exc)}"
            print(error_msg)
            raise Exception(error_msg) from exc


# Helper function to convert Anthropic API objects to dictionaries
def convert_anthropic_content_to_dict(content_item) -> Dict[str, Any]:
    """
    Convert Anthropic API content objects to dictionaries for Pydantic validation.

    Args:
        content_item: TextBlock or ToolUseBlock from Anthropic API

    Returns:
        Dict[str, Any]: Dictionary representation of the content
    """
    if hasattr(content_item, 'type'):
        if content_item.type == 'text':
            return {
                'type': 'text',
                'text': content_item.text
            }
        elif content_item.type == 'tool_use':
            return {
                'type': 'tool_use',
                'id': content_item.id,
                'name': content_item.name,
                'input': content_item.input
            }

    # Fallback: try to convert to dict if it has model_dump method
    if hasattr(content_item, 'model_dump'):
        return content_item.model_dump()
    elif hasattr(content_item, 'dict'):
        return content_item.dict()
    else:
        # Last resort: convert attributes to dict
        return {key: getattr(content_item, key) for key in dir(content_item)
                if not key.startswith('_') and not callable(getattr(content_item, key))}


# Main Chat Manager Class
class PydanticMultiToolChatManager:
    """
    Enhanced manager class with Pydantic validation for conversations with Claude API.

    This class provides comprehensive type safety, input validation, and structured
    outputs for managing multi-tool conversations with Claude.
    """

    def __init__(self, client: Any, config: Optional[ChatManagerConfig] = None) -> None:
        """
        Initialize the chat manager with validated configuration.

        Args:
            client: The Claude API client instance
            config: Optional configuration object, defaults to ChatManagerConfig()
        """
        self.client = client
        self.config = config or ChatManagerConfig()
        self.conversation_history = ConversationHistory()
        self.tool_handlers: Dict[str, ToolHandler] = {}

    def register_tool_handler(self, tool_name: str, handler: ToolHandler) -> None:
        """
        Register a handler for a specific tool with validation.

        Args:
            tool_name: The name of the tool to handle
            handler: The ToolHandler instance to process this tool

        Raises:
            ValueError: If tool doesn't exist in configuration
        """
        # Validate that the tool exists in the configuration
        tool_exists = any(tool.name == tool_name for tool in self.config.tools)
        if not tool_exists:
            raise ValueError(f"Tool '{tool_name}' not found in configuration. "
                           f"Available tools: {[t.name for t in self.config.tools]}")

        self.tool_handlers[tool_name] = handler
        print(f"Registered handler for tool: {tool_name}")

    def get_registered_tools(self) -> List[str]:
        """
        Get a list of all registered tool names.

        Returns:
            List[str]: Names of all registered tools
        """
        return list(self.tool_handlers.keys())

    def add_tool_definition(self, tool_definition: ToolDefinition) -> None:
        """
        Add a validated tool definition to the available tools.

        Args:
            tool_definition: The validated tool definition to add

        Raises:
            ValueError: If tool definition validation fails
        """
        try:
            # Remove any existing tool with the same name
            self.config.tools = [t for t in self.config.tools if t.name != tool_definition.name]
            self.config.tools.append(tool_definition)
            print(f"Added tool definition: {tool_definition.name}")
        except ValidationError as exc:
            raise ValueError(f"Invalid tool definition: {exc}") from exc

    def remove_tool(self, tool_name: str) -> bool:
        """
        Remove a tool handler and its definition.

        Args:
            tool_name: The name of the tool to remove

        Returns:
            bool: True if tool was removed, False if tool was not found
        """
        removed_handler = tool_name in self.tool_handlers
        removed_definition = False

        if removed_handler:
            del self.tool_handlers[tool_name]

        # Remove from tools list
        original_count = len(self.config.tools)
        self.config.tools = [tool for tool in self.config.tools if tool.name != tool_name]
        removed_definition = len(self.config.tools) < original_count

        if removed_handler or removed_definition:
            print(f"Removed tool: {tool_name}")
            return True
        return False

    def clear_history(self) -> None:
        """Clear the entire conversation history and reset all counters."""
        self.conversation_history.clear()
        print("Conversation history cleared")

    def get_conversation_summary(self) -> Dict[str, Any]:
        """
        Get a comprehensive summary of the conversation with metrics.

        Returns:
            Dict[str, Any]: Summary containing message count, tool calls, and other metrics
        """
        return {
            "message_count": len(self.conversation_history.messages),
            "tool_calls": self.conversation_history.tool_call_count,
            "created_at": self.conversation_history.created_at,
            "registered_tools": list(self.tool_handlers.keys()),
            "available_tools": [tool.name for tool in self.config.tools],
            "config": {
                "model": self.config.model,
                "max_tokens": self.config.max_tokens,
                "max_tool_iterations": self.config.max_tool_iterations
            }
        }

    def add_system_message(self, system_content: str) -> None:
        """
        Add a system message to the conversation history.

        Args:
            system_content: The content of the system message

        Raises:
            ValidationError: If system message validation fails
        """
        try:
            system_message = ConversationMessage(
                role="system",
                content=system_content
            )

            # Insert system message at the beginning if messages exist
            if self.conversation_history.messages:
                self.conversation_history.messages.insert(0, system_message)
            else:
                self.conversation_history.add_message(system_message)

            print("System message added to conversation")
        except ValidationError as exc:
            raise ValidationError(f"Invalid system message: {exc}") from exc

    def invoke_chat(self, question: str, print_response: bool = True) -> ChatResponse:
        """
        Send a question to Claude and handle any tool use requests with full validation.

        Args:
            question: The question or message to send to Claude
            print_response: Whether to print the response to stdout

        Returns:
            ChatResponse: Structured response with content, metrics, and status

        Raises:
            ValidationError: If message validation fails
        """
        start_time = time.time()
        initial_tool_count = self.conversation_history.tool_call_count

        try:
            # Create and validate user message
            user_message = ConversationMessage(
                role="user",
                content=question
            )
            self.conversation_history.add_message(user_message)

            # Prepare messages for API call (exclude timestamp for API compatibility)
            api_messages = [
                msg.model_dump(exclude={'timestamp'})
                for msg in self.conversation_history.messages
            ]

            # Send request to Claude
            response = self.client.messages.create(
                model=self.config.model,
                messages=api_messages,
                max_tokens=self.config.max_tokens,
                tools=[tool.model_dump() for tool in self.config.tools]
            )

            # Handle tool use iterations with configured maximum
            iteration_count = 0
            while (response.stop_reason == "tool_use" and
                   iteration_count < self.config.max_tool_iterations):
                response = self._handle_tool_use_validated(response)
                iteration_count += 1

            if iteration_count >= self.config.max_tool_iterations:
                print(f"Warning: Reached maximum tool iterations ({self.config.max_tool_iterations})")

            # Extract final response text
            final_text = response.content[0].text

            # Add Claude's final response to history (convert API objects to dicts)
            converted_content = [convert_anthropic_content_to_dict(item) for item in response.content]
            assistant_message = ConversationMessage(
                role="assistant",
                content=converted_content
            )
            self.conversation_history.add_message(assistant_message)

            # Calculate response metrics
            response_time = time.time() - start_time
            tools_used = self.conversation_history.tool_call_count - initial_tool_count

            if print_response:
                if tools_used > 0:
                    print(f"Claude's response (used {tools_used} tool(s) in {iteration_count} iteration(s)):")
                else:
                    print("Claude's response (no tools used):")
                print(final_text)

            return ChatResponse(
                content=final_text,
                tool_calls_made=tools_used,
                success=True,
                response_time=response_time,
                tokens_used=getattr(response.usage, 'total_tokens', None)
            )

        except ValidationError as exc:
            error_msg = f"Validation error: {exc}"
            print(error_msg)
            return ChatResponse(
                content="",
                tool_calls_made=0,
                success=False,
                error_message=error_msg,
                response_time=time.time() - start_time
            )
        except Exception as exc:
            error_msg = f"Unexpected error: {exc}"
            print(error_msg)
            return ChatResponse(
                content="",
                tool_calls_made=0,
                success=False,
                error_message=error_msg,
                response_time=time.time() - start_time
            )

    def _handle_tool_use_validated(self, response: Any) -> Any:
        """
        Handle tool use requests with comprehensive validation and error handling.

        Args:
            response: The response object from Claude API containing tool use request

        Returns:
            Any: The response from Claude after tool execution

        Raises:
            ValueError: If tool handler is not registered
            ValidationError: If message validation fails
        """
        # Extract tool use information
        tool_use = response.content[-1]  # Get the last content item (tool use)
        tool_name: str = tool_use.name
        tool_input: Dict[str, Any] = tool_use.input

        # Increment tool call counter
        self.conversation_history.tool_call_count += 1

        print(f"Claude wants to use tool: {tool_name} (Call #{self.conversation_history.tool_call_count})")

        # Add Claude's tool use request to history (convert API objects to dicts)
        converted_content = [convert_anthropic_content_to_dict(item) for item in response.content]
        assistant_message = ConversationMessage(
            role="assistant",
            content=converted_content
        )
        self.conversation_history.add_message(assistant_message)

        # Validate that we have a handler for this tool
        if tool_name not in self.tool_handlers:
            available_tools = list(self.tool_handlers.keys())
            raise ValueError(
                f"No handler registered for tool: {tool_name}. "
                f"Available tools: {available_tools}"
            )

        # Execute the tool with comprehensive error handling
        try:
            tool_result = self.tool_handlers[tool_name].handle(tool_input)
            print(f"Tool {tool_name} executed successfully!")
        except ValidationError as exc:
            tool_result = f"Tool input validation error: {exc}"
            print(f"Validation error in tool {tool_name}: {exc}")
        except Exception as exc:
            tool_result = f"Tool execution error: {exc}"
            print(f"Execution error in tool {tool_name}: {exc}")

        # Create validated tool response
        tool_response_content = ToolResultContent(
            tool_use_id=tool_use.id,
            content=tool_result
        )

        tool_response_message = ConversationMessage(
            role="user",
            content=[tool_response_content.model_dump()]
        )
        self.conversation_history.add_message(tool_response_message)

        # Prepare messages for next API call
        api_messages = [
            msg.model_dump(exclude={'timestamp'})
            for msg in self.conversation_history.messages
        ]

        # Send tool result back to Claude for final processing
        final_response = self.client.messages.create(
            model=self.config.model,
            messages=api_messages,
            max_tokens=self.config.max_tokens,
            tools=[tool.model_dump() for tool in self.config.tools]
        )

        return final_response


# Predefined tool definitions with proper Pydantic validation
ARTICLE_SEARCH_TOOL = ToolDefinition(
    name="get_article",
    description="Search for and retrieve Wikipedia articles by search term",
    input_schema={
        "type": "object",
        "properties": {
            "search_term": {
                "type": "string",
                "description": "The term to search for in Wikipedia",
                "minLength": 1,
                "maxLength": 100
            }
        },
        "required": ["search_term"]
    }
)

WEB_SEARCH_TOOL = ToolDefinition(
    name="web_search",
    description="A tool to retrieve up-to-date information on a given topic by searching the web",
    input_schema={
        "type": "object",
        "properties": {
            "topic": {
                "type": "string",
                "description": "The topic to search the web for",
                "minLength": 1,
                "maxLength": 100
            }
        },
        "required": ["topic"]
    }
)


# Factory Functions for Easy Setup
def create_multi_tool_chat_manager(
    client: Any,
    get_article_func: Optional[Callable[[str], str]] = None,
    web_search_func: Optional[Callable[[str], str]] = None,
    include_article_search: bool = True,
    include_web_search: bool = True,
    config: Optional[ChatManagerConfig] = None
) -> PydanticMultiToolChatManager:
    """
    Create a configured PydanticMultiToolChatManager with multiple tools.

    Args:
        client: The Claude API client instance
        get_article_func: Optional function to retrieve articles by search term
        web_search_func: Optional function to perform web searches by topic
        include_article_search: Whether to include article search tool
        include_web_search: Whether to include web search tool
        config: Optional configuration object

    Returns:
        PydanticMultiToolChatManager: Configured chat manager with requested tools

    Raises:
        ValueError: If a tool is requested but its function is not provided
    """
    # Use provided config or create default
    manager_config = config or ChatManagerConfig()

    # Add requested tools to configuration
    if include_article_search:
        if get_article_func is None:
            raise ValueError("get_article_func must be provided when include_article_search=True")
        manager_config.tools.append(ARTICLE_SEARCH_TOOL)

    if include_web_search:
        if web_search_func is None:
            raise ValueError("web_search_func must be provided when include_web_search=True")
        manager_config.tools.append(WEB_SEARCH_TOOL)

    # Create the chat manager with validated configuration
    chat_manager = PydanticMultiToolChatManager(
        client=client,
        config=manager_config
    )

    # Register tool handlers
    if include_article_search and get_article_func:
        article_handler = ArticleSearchHandler(get_article_func)
        chat_manager.register_tool_handler("get_article", article_handler)

    if include_web_search and web_search_func:
        web_handler = WebSearchHandler(web_search_func)
        chat_manager.register_tool_handler("web_search", web_handler)

    tool_count = len([t for t in [include_article_search, include_web_search] if t])
    print(f"Multi-tool chat manager created with {tool_count} tool(s)")

    return chat_manager


def create_article_only_chat_manager(
    client: Any,
    get_article_func: Callable[[str], str],
    config: Optional[ChatManagerConfig] = None
) -> PydanticMultiToolChatManager:
    """
    Create a chat manager with only article search capability.

    Args:
        client: The Claude API client instance
        get_article_func: Function to retrieve articles by search term
        config: Optional configuration object

    Returns:
        PydanticMultiToolChatManager: Chat manager with article search tool only
    """
    return create_multi_tool_chat_manager(
        client=client,
        get_article_func=get_article_func,
        include_web_search=False,
        config=config
    )


def create_web_only_chat_manager(
    client: Any,
    web_search_func: Callable[[str], str],
    config: Optional[ChatManagerConfig] = None
) -> PydanticMultiToolChatManager:
    """
    Create a chat manager with only web search capability.

    Args:
        client: The Claude API client instance
        web_search_func: Function to perform web searches by topic
        config: Optional configuration object

    Returns:
        PydanticMultiToolChatManager: Chat manager with web search tool only
    """
    return create_multi_tool_chat_manager(
        client=client,
        web_search_func=web_search_func,
        include_article_search=False,
        config=config
    )


# Example usage and testing
if __name__ == "__main__":
    # Example usage (requires anthropic client)
    print("Pydantic Multi-Tool Chat Manager")
    print("=" * 40)

    # Example tool functions
    def example_get_article(search_term: str) -> str:
        """Example article retrieval function."""
        return f"Mock Wikipedia article content for '{search_term}'"

    def example_web_search(topic: str) -> str:
        """Example web search function."""
        return f"Mock web search results for '{topic}'"

    # Create configuration
    example_config = ChatManagerConfig(
        model="claude-sonnet-4-20250514",
        max_tokens=1500,
        max_tool_iterations=3
    )

    print(f"Configuration created: {example_config.model}")
    print(f"Available tool definitions: {len([ARTICLE_SEARCH_TOOL, WEB_SEARCH_TOOL])}")
    print("\nReady for use with Anthropic client!")

Pydantic Multi-Tool Chat Manager
Configuration created: claude-sonnet-4-20250514
Available tool definitions: 2

Ready for use with Anthropic client!


## Define Tools

Define custom tool: `get_article` a function to search items from wikipedia

In [4]:
pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=ad2dbb05e3b56e69264107a385f68fdb0e875033f06ac81366d9a7dd3e0f0062
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [5]:
"""Module for retrieving Wikipedia article content."""

import wikipedia


def get_article(search_term: str) -> str:
    """
    Retrieve the content of a Wikipedia article based on a search term.

    Args:
        search_term (str): The term to search for on Wikipedia.

    Returns:
        str: The full text content of the Wikipedia article.

    Raises:
        IndexError: If no search results are found.
        wikipedia.exceptions.DisambiguationError: If the search term is ambiguous.
        wikipedia.exceptions.PageError: If the page does not exist.
    """
    # Search Wikipedia for articles matching the search term
    results = wikipedia.search(search_term)

    # Get the first result from the search
    first_result = results[0]

    # Retrieve the Wikipedia page object for the first result
    page = wikipedia.page(first_result, auto_suggest=False)

    # Return the full text content of the article
    return page.content

In [6]:
pip install serpapi google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=67f8c324d23d924a333ef6c89fb7495d9166209d505da98fb3aee63d16c6791b
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


In [ ]:
SERPAPI_API_KEY

In [7]:
from google.colab import userdata
from serpapi import GoogleSearch
from typing import List, Dict, Any

def search_serpapi(query: str) -> List[Dict[str, Any]]:
    """
    Search using SerpAPI for the given query and return the results.

    :param query: The search query string.
    :return: A list of search results.
    :raises Exception: For any errors during the request.
    """
    SERPAPI_API_KEY = userdata.get('SERPAPI_API_KEY')
    try:
        search = GoogleSearch({
            "q": query,
            "location": "New York, NY, United States",
            "api_key": SERPAPI_API_KEY
        })
        raw_results = search.get_dict()
        results = raw_results.get("organic_results", [])
        snippet = ' '.join([results[i]['snippet'] for i in range(len(results))])
        return snippet
    except Exception as e:
        raise Exception(f"An error occurred: {e}")

## Define Client

In [8]:
from anthropic import Anthropic

client = Anthropic(api_key=CLAUDE_API_KEY)

## Invoke

In [ ]:
# Create manager with validation
manager = create_multi_tool_chat_manager(
    client=client,
    get_article_func=get_article,
    web_search_func=search_serpapi
)

# Get structured response
response = manager.invoke_chat("Tell me about Python programming")
print(f"Success: {response.success}")
print(f"Tools used: {response.tool_calls_made}")
print(f"Response time: {response.response_time:.2f}s")

Registered handler for tool: get_article
Registered handler for tool: web_search
Multi-tool chat manager created with 2 tool(s)
Claude wants to use tool: get_article (Call #1)
Invoking tool: ... wait ... Searching for article: Python programming language
Tool get_article executed successfully!
Claude's response (used 1 tool(s) in 1 iteration(s)):
Based on the comprehensive information from Wikipedia, here's an overview of Python programming:

## What is Python?

Python is a **high-level, general-purpose programming language** that emphasizes code readability and simplicity. It was created by **Guido van Rossum** in the late 1980s and first released in 1991. The name comes from the British comedy group "Monty Python," reflecting the creator's desire to make programming fun.

## Key Characteristics

### **Design Philosophy**
- **Readability**: Clean, English-like syntax
- **Simplicity**: "There should be one obvious way to do it"
- **Explicit is better than implicit**
- Uses indentation 

In [18]:
print(response.content)

Based on the comprehensive information from Wikipedia, here's an overview of Python programming:

## What is Python?

Python is a **high-level, general-purpose programming language** that emphasizes code readability and simplicity. It was created by **Guido van Rossum** in the late 1980s and first released in 1991. The name comes from the British comedy group "Monty Python," reflecting the creator's desire to make programming fun.

## Key Characteristics

### **Design Philosophy**
- **Readability**: Clean, English-like syntax
- **Simplicity**: "There should be one obvious way to do it"
- **Explicit is better than implicit**
- Uses indentation (whitespace) instead of curly braces to define code blocks

### **Technical Features**
- **Dynamically typed**: Variables don't need explicit type declarations
- **Interpreted language**: Code is executed line by line
- **Garbage collected**: Automatic memory management
- **Multi-paradigm**: Supports object-oriented, procedural, and functional pro

## Continuous Chat Interface

In [11]:
while True:
    user_input = input("> You: ")

    if user_input.lower() == 'exit' or user_input.lower() == 'quit':
        break

    # Create manager with validation
    manager = create_multi_tool_chat_manager(
        client=client,
        get_article_func=get_article,
        web_search_func=search_serpapi
    )

    # Get structured response
    response = manager.invoke_chat(user_input)
    print(f"Success: {response.success}")
    print(f"Tools used: {response.tool_calls_made}")
    print(f"Response time: {response.response_time:.2f}s")

    # Bot response
    print(f"> Claude's response: {response.content}")

> You: news about tesla robotaxi
Registered handler for tool: get_article
Registered handler for tool: web_search
Multi-tool chat manager created with 2 tool(s)
Claude wants to use tool: web_search (Call #1)
Invoking tool: ... wait ... Searching the web for: Tesla robotaxi news
Tool web_search executed successfully!
Claude wants to use tool: web_search (Call #2)
Invoking tool: ... wait ... Searching the web for: Tesla robotaxi 2024 Austin launch Cybercab
Tool web_search executed successfully!
Claude's response (used 2 tool(s) in 2 iteration(s)):
Based on the latest information, here are the key updates about Tesla's robotaxi program:

## Tesla Robotaxi News Summary

### Current Operations (Austin Launch - June 2024)
- **Tesla launched its robotaxi service in Austin, Texas** in June 2024 as a limited pilot program
- **Using Model Y vehicles** (not the Cybercab) with safety drivers in the passenger seat
- **Limited area operations** with about 10 cars initially
- **Supervised rides** - i